In [0]:
import cv2
import numpy as np
from tqdm import tqdm
import os
from keras.utils import to_categorical
#loading dataset from drive
dir_name = '/content/drive/My Drive/ML_Datasets/Mini_Merced/'
classes = dict()
x = 0
X = []
Y = []

#creating dataset
for d in os.listdir(dir_name):
    print(d)
    classes.update({d:x})
    x += 1

    for img in os.listdir(dir_name+'/'+d):
        
        im = cv2.imread(dir_name+'/'+d+'/'+img,1)
        
        im = cv2.resize(im,(224,224),interpolation = cv2.INTER_LINEAR)
        X.append(np.array(im))
        Y.append(classes[d])

print(classes)
X = np.array(X)
Y = np.array(Y)
Y = np.reshape(Y,(-1,1))
Y = to_categorical(Y,dtype=int)
#saving numpy array of dataset
np.save('X.npy',X)
np.save('Y.npy',Y)
print(X.shape,Y.shape)
#print(classes)
print(X[:2])
print(Y[:2])
        

In [0]:
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.models import Model,Sequential
from keras.layers import Dense, GlobalAveragePooling2D,Dropout
from keras import backend as K
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split

#loading the dataset
X = np.load('X.npy')
Y = np.load('Y.npy')
n_classes = 6
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2, random_state=0)
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
X = X.astype('float32')
train_X /= 255
X /= 255
test_X /= 255


print(train_X.shape,test_X.shape,train_Y.shape,test_Y.shape)
base_model = VGG16(weights = 'imagenet')
base_model.summary()
#printing the weights of layer 10 before fine tuning
print(base_model.layers[10].get_weights())

#setting the first 10 layers as non-trainable
for layer in base_model.layers[:5]:
    layer.trainable = False
    
epochs = 2
batch_size = 32

#creating the model
model = Sequential()
model.add(base_model)
model.add(Dropout(0.7))
model.add(Dense(n_classes,activation='softmax'))

#model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X,Y,epochs = epochs,batch_size = batch_size,validation_data = (test_X,test_Y))
scores = model.evaluate(test_X,test_Y,verbose = 0)
print("Accuracy: %.4f%%" % (scores[1]*100))

#printing the weights after fine tuning
print(base_model.layers[10].get_weights())





In [0]:
#saving the features for each input image
new_X = []
base_model = VGG16(weights = 'imagenet',include_top = False)
for x in X:
    n_x = np.reshape(x,(1,224,224,3))
    new_X.append(base_model.predict_on_batch(n_x))
  
new_X = np.array(new_X)
print(new_X.shape)
new_X = np.reshape(new_X,(600,7*7*512))

In [0]:
import numpy as np
from keras.layers import Dense,Input
from keras.datasets import mnist
from keras.models import Model,Sequential,load_model
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import matplotlib.pyplot as plt

train_X, test_X, train_Y, test_Y = train_test_split(new_X, Y, test_size=0.3, random_state=0)

train_X = train_X/255
test_X = test_X/255



#creating stacked auto-encoder haing 3 auto-encoders

input_img = Input(shape=(25088,))
encode = Dense(1024,activation='relu')(input_img)
output_img = Dense(25088,activation='sigmoid')(encode)
encoder_1 = Model(input_img,encode)
auto_1 = Model(input_img,output_img)
auto_1.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
auto_1.fit(train_X, train_X, epochs=5,batch_size=256,validation_data=(test_X,test_X))


encoder_1.save('Encoder_1')
in_1 = encoder_1.predict(train_X)

input_img = Input(shape=(in_1.shape[1],))
encode = Dense(512,activation='relu')(input_img)
output_img = Dense(in_1.shape[1],activation='sigmoid')(encode)
encoder_2 = Model(input_img,encode)
auto_2 = Model(input_img,output_img)
auto_2.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])  
auto_2.fit(in_1, in_1, epochs=5,batch_size=256)

encoder_2.save('Encoder_2')
in_2 = encoder_2.predict(in_1)

input_img = Input(shape=(in_2.shape[1],))
encode = Dense(256,activation='relu')(input_img)
output_img = Dense(in_2.shape[1],activation='sigmoid')(encode)
encoder_3 = Model(input_img,encode)
auto_3 = Model(input_img,output_img)
auto_3.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])  
auto_3.fit(in_2, in_2, epochs=5,batch_size=256)
in_3 = encoder_3.predict(in_2)
print(in_3.shape)

np.save('encoding',in_3)

encoder_3.save('Encoder_3')


#creating the model of auto-encoders for predicting
model = Sequential()
x = load_model('Encoder_1')
model.add(x)
x = load_model('Encoder_2')
model.add(x)
x = load_model('Encoder_3')
model.add(x)

model.add(Dense(6,activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])  
model.fit(train_X, train_Y, epochs=5,batch_size=128,validation_data = (test_X,test_Y))

#predictions and confusion_matrix
pred = model.predict(test_X)
pred = pred.argmax(axis=1)
c_matrix = confusion_matrix(test_Y.argmax(axis=1),pred)
print(c_matrix)

#accuracy
accuracy = accuracy_score(test_Y.argmax(axis=1),pred)
print('Accuracy : ',accuracy)

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

#creating svm classifier
# clf.fit(in_3, train_Y.argmax(axis=1))
clf = OneVsRestClassifier(LinearSVC(random_state=42)).fit(in_3,train_Y.argmax(axis=1) )
in_1 = encoder_1.predict(test_X)
in_2 = encoder_2.predict(in_1)
in_3 = encoder_3.predict(in_2)
pred = clf.predict(in_3)

#predictions and confusion_matrix  
c_matrix = confusion_matrix(test_Y.argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(test_Y.argmax(axis=1),pred)
print('Accuracy : ',accuracy)









